In [1]:
import pandas as pd
import scipy.io as sio

mat_db = sio.loadmat('B0005.mat') ['B0005']

FileNotFoundError: [Errno 2] No such file or directory: 'B0005.mat'

In [0]:
def to_df(mat_db):
    """Returns one pd.DataFrame per cycle type"""

    # Features common for every cycle
    cycles_cols = ['type', 'ambient_temperature', 'time']

    # Features monitored during the cycle
    features_cols = {
        'charge': ['Voltage_measured', 'Current_measured', 'Temperature_measured', 
                'Current_charge', 'Voltage_charge', 'Time'],
        'discharge': ['Voltage_measured', 'Current_measured', 'Temperature_measured', 
                    'Current_charge', 'Voltage_charge', 'Time', 'Capacity'],
        'impedance': ['Sense_current', 'Battery_current', 'Current_ratio',
                    'Battery_impedance', 'Rectified_impedance', 'Re', 'Rct']
    }

    # Define one pd.DataFrame per cycle type
    df = {key: pd.DataFrame() for key in features_cols.keys()}

    # Get every cycle
    print(f'Number of cycles: {mat_db[0][0][0].shape[1]}')
    cycles = [[row.flat[0] for row in line] for line in mat_db[0][0][0][0]]

    # Get measures for every cycle
    for cycle_id, cycle_data in enumerate(cycles):
        tmp = pd.DataFrame()

        # Data series for every cycle
        features_x_cycle = cycle_data[-1]

        # Get features for the specific cycle type
        features = features_cols[cycle_data[0]]
        
        for feature, data in zip(features, features_x_cycle):
            if len(data[0]) > 1:
                # Correct number of records
                tmp[feature] = data[0]
            else:
                # Single value, so assign it to all rows
                tmp[feature] = data[0][0]
        
        # Add columns common to the cycle measurements
        tmp['id_cycle'] = cycle_id
        for k, col in enumerate(cycles_cols):
            tmp[col] = cycle_data[k]
        
        # Append cycle data to the right pd.DataFrame
        cycle_type = cycle_data[0]
        df[cycle_type] = df[cycle_type].append(tmp, ignore_index=True)
    
    return df

In [17]:
dfs = to_df(mat_db)

dfs['charge'].head()

Number of cycles tested: 616


,Voltage_measured,Current_measured,Temperature_measured,Current_charge,Voltage_charge,Time,id_cycle,type,ambient_temperature,time
0,3.873017,-0.001201,24.655358,0.000,0.003,0.000,0,charge,24,2008.0
1,3.479394,-4.030268,24.666480,-4.036,1.570,2.532,0,charge,24,2008.0
2,4.000588,1.512731,24.675394,1.500,4.726,5.500,0,charge,24,2008.0
3,4.012395,1.509063,24.693865,1.500,4.742,8.344,0,charge,24,2008.0
4,4.019708,1.511318,24.705069,1.500,4.753,11.125,0,charge,24,2008.0
